In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # INFO, WARNING messages are not printed
import tensorflow as tf
import time # for throughput measurements

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
print('Number of GPUs available :', len(gpus))
if gpus:
    gpu_num = 0 # Number of the GPU to be used
    try:
        #tf.config.set_visible_devices([], 'GPU')
        tf.config.set_visible_devices(gpus[gpu_num], 'GPU')
        print('Only GPU number', gpu_num, 'used.')
        tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
    except RuntimeError as e:
        print(e)

# uninstall sionna first
import sys
from pathlib import Path

sys.path.append(str(Path('..')))

from sionna.fec.ldpc import QLDPCBPDecoder, readAlist, Feedback_GNN, GNN_BP4, save_weights, load_weights 
from sionna.fec.ldpc import Sandwich_BP_GNN_Evaluation_Model, First_Stage_BP_Model, Second_Stage_GNN_BP_Model
from sionna.fec.ldpc import *
from sionna.utils import BinarySource 
from sionna.utils.metrics import count_block_errors
from sionna.channel import Pauli
from sionna.utils.plotting import PlotBER

from tensorflow.keras.losses import BinaryCrossentropy
from sionna.utils.metrics import compute_bler
from sionna.fec.utils import int_mod_2

from tqdm.notebook import tqdm

Number of GPUs available : 1
Only GPU number 0 used.


# Train feedback GNN on mixed data (hard samples repeat 50 times) for the [[1270,28]] code.

In [4]:
GHP_n1270_k28 = create_QC_GHP_codes(127, np.array([[0,-1,51,52,-1],[-1,0,-1,111,20],[0,-1,98,-1,122],[0,80,-1,119,-1],[-1,0,5,-1,106]]), [0,1,7], name="GHP_n1270_k28") # 16 <= d <= 46
code = GHP_n1270_k28
num_iter1 = tf.constant(64)
num_iter2 = tf.constant(16)

factor1 = tf.constant(1.0)
factor2 = tf.constant(1.0)

decoder1 = QLDPCBPDecoder(code=code, num_iter=num_iter1, normalization_factor=factor1, cn_type="boxplus-phi", trainable=False, stage_one=True)
decoder2 = QLDPCBPDecoder(code=code, num_iter=num_iter2, normalization_factor=factor2, cn_type="boxplus-phi", trainable=False, stage_two=True)
# stage-two BP decoder does some extra calculations for multi-loss calculation

bs = tf.constant(100)
n = tf.constant(code.N)
cn_z = tf.constant(code.hz.shape[0])
cn_x = tf.constant(code.hx.shape[0])

G = Feedback_GNN(code=code, 
                 num_msg_dims=tf.constant(20),
                 num_hidden_units=tf.constant(40),
                 num_mlp_layers=2,
                 reduce_op="mean",      # can choose from [sum, mean, max, min]
                 activation="tanh",
                 use_bias=True)

# Split into two models so that the first stage BP decoding can be done in XLA mode.
model_stage_one  = First_Stage_BP_Model(code, decoder1)
# The implementation of second stage involves TensorArray, taking gradient is not supported by XLA.
model_stage_two = Second_Stage_GNN_BP_Model(code, G, decoder2, num_iter=num_iter2)

dataset_x = tf.data.Dataset.from_tensor_slices(np.load("../sionna/fec/ldpc/datasets/n1270_k28_wt_10_80_x_all.npy"))
dataset_z = tf.data.Dataset.from_tensor_slices(np.load("../sionna/fec/ldpc/datasets/n1270_k28_wt_10_80_z_all.npy"))                                              


dataset = tf.data.Dataset.zip((dataset_x, dataset_z))
dataset_size = dataset.cardinality()
print("datase size", dataset_size)
dataset = dataset.shuffle(dataset_size, reshuffle_each_iteration=True)
bs      = 100
repeat  = 1
dataset = dataset.repeat(repeat).batch(bs)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


it = tf.constant(0)
decay_steps = int(dataset_size * repeat / bs) + 1
# We only train for one epoch. If train multiple epochs, better use cosine scheduing.
# decayed_lr = tf.keras.optimizers.schedules.CosineDecay(2e-4, decay_steps)
# optimizer = tf.keras.optimizers.Adam(decayed_lr)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4) 

clip_value_grad = 10 # gradient clipping for stable training convergence
for x,z in tqdm(dataset):
    it += 1
    h_vn, logit_hx_perp, logit_hz_perp = model_stage_one(x, z)
    with tf.GradientTape() as tape:
        s_hat, b_hat, loss = model_stage_two(x, z, h_vn, logit_hx_perp, logit_hz_perp)
        
    flagged_bler = compute_bler(tf.zeros_like(s_hat), s_hat)
    bler = compute_bler(tf.zeros_like(b_hat), b_hat)
    if it % 500 == 0:
        print(f"Iteration {it}/{decay_steps}. Current loss: {loss:3f} bler: {bler:.4f} flagged bler: {flagged_bler:.4f}".format())

    grads = tape.gradient(loss, model_stage_two.trainable_variables)
    grads = [tf.clip_by_value(grad, -clip_value_grad, clip_value_grad) for grad in grads]
    optimizer.apply_gradients(zip(grads, model_stage_two.trainable_weights))    

save_weights(G, f"../sionna/fec/ldpc/weights/feedback_GNN_n1270_k28_wt_10_80_iter_64_16_mixed.npy") # change name to prevent overwriting


datase size tf.Tensor(1307084, shape=(), dtype=int64)


  0%|          | 0/13071 [00:00<?, ?it/s]

Iteration 500/13071. Current loss: 0.012896 bler: 0.0300 flagged bler: 0.0300
Iteration 1000/13071. Current loss: 0.011347 bler: 0.0300 flagged bler: 0.0300
Iteration 1500/13071. Current loss: 0.007706 bler: 0.0200 flagged bler: 0.0200
Iteration 2000/13071. Current loss: 0.012872 bler: 0.0200 flagged bler: 0.0200
Iteration 2500/13071. Current loss: 0.004619 bler: 0.0100 flagged bler: 0.0100
Iteration 3000/13071. Current loss: 0.004173 bler: 0.0000 flagged bler: 0.0000
Iteration 3500/13071. Current loss: 0.000772 bler: 0.0000 flagged bler: 0.0000
Iteration 4000/13071. Current loss: 0.010500 bler: 0.0100 flagged bler: 0.0100
Iteration 4500/13071. Current loss: 0.009045 bler: 0.0300 flagged bler: 0.0300
Iteration 5000/13071. Current loss: 0.003627 bler: 0.0100 flagged bler: 0.0100
Iteration 5500/13071. Current loss: 0.004644 bler: 0.0000 flagged bler: 0.0000
Iteration 6000/13071. Current loss: 0.002655 bler: 0.0000 flagged bler: 0.0000
Iteration 6500/13071. Current loss: 0.003415 bler: 0.

# Load the trained GNN model and evaluate it.

In [7]:
GHP_n1270_k28 = create_QC_GHP_codes(127, np.array([[0,-1,51,52,-1],[-1,0,-1,111,20],[0,-1,98,-1,122],[0,80,-1,119,-1],[-1,0,5,-1,106]]), [0,1,7], name="GHP_n1270_k28") # 16 <= d <= 46
code = GHP_n1270_k28

ber_plot = PlotBER()

bs = tf.constant(5000)
n = tf.constant(code.N)
cn_z = tf.constant(code.hz.shape[0])
cn_x = tf.constant(code.hx.shape[0])

G = Feedback_GNN(code=code, 
                 num_msg_dims=tf.constant(20),
                 num_hidden_units=tf.constant(40),
                 num_mlp_layers=2,
                 reduce_op="mean",     
                 activation="tanh",
                 use_bias=True)
# Pass dummy input to the model first.
G((tf.zeros((bs, n, 3)), tf.zeros((cn_x, bs)), tf.zeros((cn_z, bs)), 
                  tf.zeros((cn_x, bs)), tf.zeros((cn_z, bs))))
# Then load the stored weights.
load_weights(G, f"../sionna/fec/ldpc/weights/feedback_GNN_n1270_k28_wt_10_80_iter_64_16_mixed.npy")  


In [3]:
num_iter1 = tf.constant(64)
num_iter2 = tf.constant(16)
factor1 = tf.constant(1.0)
factor2 = tf.constant(1.0)

decoder1 = QLDPCBPDecoder(code=code, num_iter=num_iter1, normalization_factor=factor1, cn_type="boxplus-phi", trainable=False, stage_one=True)
decoder2 = QLDPCBPDecoder(code=code, num_iter=num_iter2, normalization_factor=factor2, cn_type="boxplus-phi", trainable=False, stage_one=True)


model_eval = Sandwich_BP_GNN_Evaluation_Model(code, [decoder1]+[decoder2]*3, [G]*3, num_layers=4)
p = np.arange(0.10, 0.141, 0.01)[::-1]
ber_plot.simulate(model_eval,
              ebno_dbs=p,
              batch_size=bs,
              num_target_block_errors=100, # stop sim after 100 logical errors
              legend=f"feedback GNN {factor1.numpy():.2f} (G,G,G)",
              soft_estimates=True,
              max_mc_iter=6000,
              early_stop=True, # stop simulation if no error has been detected at current SNR point
              add_bler=True,   # logical error rate
              show_fig=False,  # do not show the figure after all results are simulated
              qldpc=True,      # can see number of flagged errors
              forward_keyboard_interrupt=False)

        p |    Flagged |       BLER | flag errors | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
     0.14 | 3.9640e-01 | 3.9640e-01 |        1982 |         1982 |        5000 |        12.9 |reached target block errors
     0.13 | 1.3880e-01 | 1.3880e-01 |         694 |          694 |        5000 |         0.8 |reached target block errors
     0.12 | 2.7600e-02 | 2.7600e-02 |         138 |          138 |        5000 |         0.8 |reached target block errors
     0.11 | 3.7000e-03 | 3.7000e-03 |         111 |          111 |       30000 |         4.8 |reached target block errors
      0.1 | 3.6786e-04 | 3.6786e-04 |         103 |          103 |      280000 |        44.9 |reached target block errors


(<tf.Tensor: shape=(5,), dtype=float64, numpy=
 array([3.96400000e-01, 1.38800000e-01, 2.76000000e-02, 3.70000000e-03,
        3.67857143e-04])>,
 <tf.Tensor: shape=(5,), dtype=float64, numpy=
 array([3.96400000e-01, 1.38800000e-01, 2.76000000e-02, 3.70000000e-03,
        3.67857143e-04])>)

In [4]:
model_eval.summary()

Model: "sandwich_bp_gnn__evaluation__model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 binary_source (BinarySource  multiple                 0 (unused)
 )                                                               
                                                                 
 pauli (Pauli)               multiple                  0         
                                                                 
 qldpcbp_decoder (QLDPCBPDec  multiple                 0         
 oder)                                                           
                                                                 
 qldpcbp_decoder_1 (QLDPCBPD  multiple                 0         
 ecoder)                                                         
                                                                 
 feedback_gnn (Feedback_GNN)  multiple                 3923      
                                

# [[882, 24]] 

In [5]:
GHP_n882_k24 = create_QC_GHP_codes(63, create_cyclic_permuting_matrix(7, [27,54,0]), [0,1,6]) # 18 <= d <= 24
code = GHP_n882_k24

num_iter1 = tf.constant(64)   # when training easy + hard examples
num_iter2 = tf.constant(16)
factor = tf.constant(1.0)
decoder1 = QLDPCBPDecoder(code=code, num_iter=num_iter1, normalization_factor=factor, cn_type="boxplus-phi", trainable=False, stage_one=True)
decoder2 = QLDPCBPDecoder(code=code, num_iter=num_iter2, normalization_factor=factor, cn_type="boxplus-phi", trainable=True, stage_two=True)
G = Feedback_GNN(code=code, 
                 num_msg_dims=tf.constant(20),
                 num_hidden_units=tf.constant(40),
                 num_mlp_layers=2,
                 reduce_op="mean",
                 activation="tanh",
                 use_bias=True)

model_stage_one  = First_Stage_BP_Model(code, decoder1)
model_stage_two = Second_Stage_GNN_BP_Model(code, G, decoder2, num_iter=num_iter2)

dataset_x = tf.data.Dataset.from_tensor_slices(np.load("../sionna/fec/ldpc/datasets/n882_k24_wt_4_60_x_all.npy"))
dataset_z = tf.data.Dataset.from_tensor_slices(np.load("../sionna/fec/ldpc/datasets/n882_k24_wt_4_60_z_all.npy"))  

dataset = tf.data.Dataset.zip((dataset_x, dataset_z))
dataset_size = dataset.cardinality()
print("datase size", dataset_size)
dataset = dataset.shuffle(dataset_size, reshuffle_each_iteration=True)
bs      = 100
repeat  = 1
dataset = dataset.repeat(repeat).batch(bs)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


it = tf.constant(0)
decay_steps = int(dataset_size * repeat / bs) + 1

# decayed_lr = tf.keras.optimizers.schedules.CosineDecay(1e-4, decay_steps) 
# optimizer = tf.keras.optimizers.Adam(decayed_lr)
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4)                


clip_value_grad = 10 # gradient clipping for stable training convergence
for x,z in tqdm(dataset):
    it += 1
    h_vn, logit_hx_perp, logit_hz_perp = model_stage_one(x, z)
    with tf.GradientTape() as tape:
        s_hat, b_hat, loss = model_stage_two(x, z, h_vn, logit_hx_perp, logit_hz_perp)
        
    flagged_bler = compute_bler(tf.zeros_like(s_hat), s_hat)
    bler = compute_bler(tf.zeros_like(b_hat), b_hat)
    if it % 500 == 0:
        print(f"Iteration {it}/{decay_steps}. Current loss: {loss:3f} bler: {bler:.4f} flagged bler: {flagged_bler:.4f}".format())

    grads = tape.gradient(loss, model_stage_two.trainable_variables)
    grads = [tf.clip_by_value(grad, -clip_value_grad, clip_value_grad) for grad in grads]
    optimizer.apply_gradients(zip(grads, model_stage_two.trainable_weights))    

save_weights(G, f"../sionna/fec/ldpc/weights/feedback_GNN_n882_k24_wt_4_60_iter_64_16_mixed.npy") # change name to prevent overwriting



datase size tf.Tensor(1287116, shape=(), dtype=int64)


  0%|          | 0/12872 [00:00<?, ?it/s]

Iteration 500/12872. Current loss: 0.014715 bler: 0.0100 flagged bler: 0.0100
Iteration 1000/12872. Current loss: 0.013874 bler: 0.0300 flagged bler: 0.0300
Iteration 1500/12872. Current loss: 0.006196 bler: 0.0000 flagged bler: 0.0000
Iteration 2000/12872. Current loss: 0.007878 bler: 0.0100 flagged bler: 0.0100
Iteration 2500/12872. Current loss: 0.006150 bler: 0.0100 flagged bler: 0.0100
Iteration 3000/12872. Current loss: 0.003681 bler: 0.0000 flagged bler: 0.0000
Iteration 3500/12872. Current loss: 0.004665 bler: 0.0100 flagged bler: 0.0100
Iteration 4000/12872. Current loss: 0.011738 bler: 0.0200 flagged bler: 0.0200
Iteration 4500/12872. Current loss: 0.007271 bler: 0.0200 flagged bler: 0.0200
Iteration 5000/12872. Current loss: 0.007780 bler: 0.0100 flagged bler: 0.0100
Iteration 5500/12872. Current loss: 0.005075 bler: 0.0000 flagged bler: 0.0000
Iteration 6000/12872. Current loss: 0.006377 bler: 0.0000 flagged bler: 0.0000
Iteration 6500/12872. Current loss: 0.007058 bler: 0.

In [3]:
GHP_n882_k24 = create_QC_GHP_codes(63, create_cyclic_permuting_matrix(7, [27,54,0]), [0,1,6]) # 18 <= d <= 24
code = GHP_n882_k24

ber_plot = PlotBER()

bs = tf.constant(5000)
n = tf.constant(code.N)
cn_z = tf.constant(code.hz.shape[0])
cn_x = tf.constant(code.hx.shape[0])

G = Feedback_GNN(code=code, 
                 num_msg_dims=tf.constant(20),
                 num_hidden_units=tf.constant(40),
                 num_mlp_layers=2,
                 reduce_op="mean",     
                 activation="tanh",
                 use_bias=True)
G((tf.zeros((bs, n, 3)), tf.zeros((cn_x, bs)), tf.zeros((cn_z, bs)), 
                  tf.zeros((cn_x, bs)), tf.zeros((cn_z, bs))))
load_weights(G, f"../sionna/fec/ldpc/weights/feedback_GNN_n882_k24_wt_4_60_iter_64_16_mixed.npy")   


In [4]:
num_iter1 = tf.constant(64)
num_iter2 = tf.constant(16)
factor1 = tf.constant(1.0)
factor2 = tf.constant(1.0)

decoder1 = QLDPCBPDecoder(code=code, num_iter=num_iter1, normalization_factor=factor1, cn_type="boxplus-phi", trainable=False, stage_one=True)
decoder2 = QLDPCBPDecoder(code=code, num_iter=num_iter2, normalization_factor=factor2, cn_type="boxplus-phi", trainable=False, stage_one=True)

model_eval = Sandwich_BP_GNN_Evaluation_Model(code, [decoder1]+[decoder2]*3, [G]*3, num_layers=4)
p = np.arange(0.09, 0.141, 0.01)[::-1]
ber_plot.simulate(model_eval,
              ebno_dbs=p,
              batch_size=bs,
              num_target_block_errors=100, # stop sim after 2000 bit errors
              legend=f"feedback GNN {factor1.numpy():.2f} (G,G,G)",
              soft_estimates=True,
              max_mc_iter=6000,
              early_stop=True, # stop simulation if no error has been detected at current SNR point
              add_bler=True, # in case BLER is also interesting
              show_fig=False, # we show the figure after all results are simulated
              qldpc=True,
              forward_keyboard_interrupt=False)

        p |    Flagged |       BLER | flag errors | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
     0.14 | 4.8340e-01 | 4.8340e-01 |        2417 |         2417 |        5000 |        11.5 |reached target block errors
     0.13 | 2.3360e-01 | 2.3360e-01 |        1168 |         1168 |        5000 |         0.5 |reached target block errors
     0.12 | 7.8400e-02 | 7.8400e-02 |         392 |          392 |        5000 |         0.5 |reached target block errors
     0.11 | 2.0000e-02 | 2.0000e-02 |         100 |          100 |        5000 |         0.5 |reached target block errors
      0.1 | 3.0000e-03 | 3.0000e-03 |         105 |          105 |       35000 |         3.4 |reached target block errors
     0.09 | 4.4889e-04 | 4.4889e-04 |         101 |          101 |      225000 |        21.6 |reached target block errors


(<tf.Tensor: shape=(6,), dtype=float64, numpy=
 array([4.83400000e-01, 2.33600000e-01, 7.84000000e-02, 2.00000000e-02,
        3.00000000e-03, 4.48888889e-04])>,
 <tf.Tensor: shape=(6,), dtype=float64, numpy=
 array([4.83400000e-01, 2.33600000e-01, 7.84000000e-02, 2.00000000e-02,
        3.00000000e-03, 4.48888889e-04])>)